In [4]:
# 분별 날짜 파일에 있는 날짜 기준으로 DB에서 데이터 받아와서 저장하기

import MySQLdb
import os
import numpy as np
import csv
import datetime

# MySQL DB 연결
db = MySQLdb.connect('210.102.142.13',"root", "witlab8*", "cas_db")
c = db.cursor()

# 분별 날짜
# date = np.genfromtxt('../data/date_sunrise_sunset.csv', delimiter=',', dtype='str')
date = np.genfromtxt('../data/test_date_sunrise_sunset.csv', delimiter=',', dtype='str')

# csv 파일로 내보내기
# w = open('../data/db_connect_data_am.csv', 'w', encoding='utf-8')
w = open('../data/db_connect_test_data_am.csv', 'w', encoding='utf-8')
wr = csv.writer(w)

# csv 파일로 내보내기
# w2 = open('../data/db_connect_data_pm.csv', 'w', encoding='utf-8')
w2 = open('../data/db_connect_test_data_pm.csv', 'w', encoding='utf-8')
wr2 = csv.writer(w2)

# 각 날짜별 데이터들 DB에서 가져오고 csv 파일로 저장
# swr, mwr, lwr data 받아옴
for j in range(len(date)):
    sql = "select time(date),cas_swr, cas_mwr, cas_lwr from natural_tracker left outer join cas_wave_ratio using(date) where date(date) = '"+ str(date[j][0]) + "' order by time(date)"
    c.execute(sql)
    rows = c.fetchall()
    
     # 일출 후 6시간과 일몰 후 6시간 데이터 사용, 일출 혹은 일몰 당시 데이터가 없을 경우 가장 가까운 다른 데이터로 변환
    sunrise = datetime.datetime.strptime(date[j][1], '%H:%M:%S')
    sunset = datetime.datetime.strptime(date[j][2], '%H:%M:%S')
    
    standard = datetime.datetime.strptime('1900-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
    
    sunrise = sunrise - standard
    sunset = sunset - standard
    
    start = 0
    end = 0
    
    for i in range(len(rows)):
        if(rows[i][0] >= sunrise):
            start = i
            print(date[j][0], rows[start][0], sunrise)
            break;
    
    for i in range(len(rows)):
        if(rows[i][0] >= sunset):
            end = i
            print(date[j][0], rows[end][0], sunset)
            break;

    # start에 저장된 index부터 772개 데이터 가져와서 저장
    for l in range(start, start+360):
        wr.writerow([rows[l][1], rows[l][2], rows[l][3]])
    print(date[j][0] + " am saved.")
    
    for l in range(end-360, end):
        wr2.writerow([rows[l][1], rows[l][2], rows[l][3]])
    print(date[j][0] + " pm saved.")
        
w.close()
w2.close()
db.close()

2018-04-19 5:52:00 5:52:00
2018-04-19 19:52:00 19:52:00
2018-04-19 am saved.
2018-04-19 pm saved.
2018-04-21 5:49:00 5:49:00
2018-04-21 19:49:00 19:49:00
2018-04-21 am saved.
2018-04-21 pm saved.
2018-04-26 5:43:00 5:43:00
2018-04-26 19:43:00 19:43:00
2018-04-26 am saved.
2018-04-26 pm saved.
2018-04-27 5:42:00 5:42:00
2018-04-27 19:42:00 19:42:00
2018-04-27 am saved.
2018-04-27 pm saved.
2018-04-28 5:41:00 5:41:00
2018-04-28 19:41:00 19:41:00
2018-04-28 am saved.
2018-04-28 pm saved.
2018-04-29 5:40:00 5:40:00
2018-04-29 19:40:00 19:40:00
2018-04-29 am saved.
2018-04-29 pm saved.
2018-04-30 5:38:00 5:38:00
2018-04-30 19:38:00 19:38:00
2018-04-30 am saved.
2018-04-30 pm saved.
2018-05-04 5:34:00 5:34:00
2018-05-04 19:34:00 19:34:00
2018-05-04 am saved.
2018-05-04 pm saved.
2018-05-05 5:33:00 5:33:00
2018-05-05 19:33:00 19:33:00
2018-05-05 am saved.
2018-05-05 pm saved.
2018-05-08 5:30:00 5:30:00
2018-05-08 19:30:00 19:30:00
2018-05-08 am saved.
2018-05-08 pm saved.
2018-05-10 5:28:00 5

In [5]:
# date = np.genfromtxt('../data/date_sunrise_sunset.csv', delimiter=',', dtype='str')
date = np.genfromtxt('../data/test_date_sunrise_sunset.csv', delimiter=',', dtype='str')

# import_data = np.loadtxt('../data/db_connect_data_am.csv', delimiter=',')
import_data = np.loadtxt('../data/db_connect_test_data_am.csv', delimiter=',')


one = import_data[:,0] # swr
two = import_data[:,1] # mwr
thr = import_data[:,2] # lwr

swr = []
mwr = []
lwr = []

for cnt in range(len(date)): # 전체 데이터 날짜 수 (행)
    temp = []
    temp2 = []
    temp3 = []
    for i in range(360): # 한 날짜의 데이터들 한 행에 저장 (열)
        temp.append(one[i+360*cnt])
        temp2.append(two[i+360*cnt])
        temp3.append(thr[i+360*cnt])
    swr.append(temp)
    mwr.append(temp2)
    lwr.append(temp3)

# w = open('../data/swr_ratio_calculation_am.csv', 'w', encoding='utf-8')
w = open('../data/swr_ratio_test_calculation_am.csv', 'w', encoding='utf-8')
wr = csv.writer(w)

for i in range(len(date)):
    ratio = []
    
    for j in range(6):
        grade = 0
        all_hap = 0
        swr_hap = 0
        for k in range(60):
            swr_hap += swr[i][k + 60*j]
            all_hap += swr[i][k + 60*j] + mwr[i][k + 60*j] + lwr[i][k + 60*j]
        
        ratio.append(round(swr_hap / all_hap * 100, 2))
    
    wr.writerow([ratio[0], ratio[1], ratio[2], ratio[3], ratio[4], ratio[5]])
w.close()

In [6]:
# date = np.genfromtxt('../data/date_sunrise_sunset.csv', delimiter=',', dtype='str')
date = np.genfromtxt('../data/test_date_sunrise_sunset.csv', delimiter=',', dtype='str')

# import_data = np.loadtxt('../data/db_connect_data_pm.csv', delimiter=',')
import_data = np.loadtxt('../data/db_connect_test_data_pm.csv', delimiter=',')

one = import_data[:,0] # swr
two = import_data[:,1] # mwr
thr = import_data[:,2] # lwr

swr = []
mwr = []
lwr = []

for cnt in range(len(date)): # 전체 데이터 날짜 수 (행)
    temp = []
    temp2 = []
    temp3 = []
    for i in range(360): # 한 날짜의 데이터들 한 행에 저장 (열)
        temp.append(one[i+360*cnt])
        temp2.append(two[i+360*cnt])
        temp3.append(thr[i+360*cnt])
    swr.append(temp)
    mwr.append(temp2)
    lwr.append(temp3)

# w = open('../data/swr_ratio_calculation_pm.csv', 'w', encoding='utf-8')
w = open('../data/swr_ratio_test_calculation_pm.csv', 'w', encoding='utf-8')
wr = csv.writer(w)

for i in range(len(date)):
    all_hap = 0
    swr_hap = 0
    ratio = 0
    grade = 0
    
    for j in range(len(swr[i])):
        swr_hap += swr[i][j]
        all_hap += swr[i][j] + mwr[i][j] + lwr[i][j]
    
    ratio = round(swr_hap / all_hap * 100, 2)
    
    if(ratio >= 19.00):
        grade += 1
    if(ratio >= 20.00):
        grade += 1
    if(ratio >= 22.00):
        grade += 1
    if(ratio >= 23.00):
        grade += 1
    
    wr.writerow([grade])
w.close()

In [7]:
import_data = np.genfromtxt('../data/swr_ratio_calculation_am.csv', delimiter=',', dtype='float')
import_data2 = np.genfromtxt('../data/swr_ratio_test_calculation_am.csv', delimiter=',', dtype='float')
w = open('../data/swr_ratio_classification_input_am.csv', 'w', encoding='utf-8')
wr = csv.writer(w)

for i in range(len(import_data)):
    wr.writerow(import_data[i][:])
for i in range(len(import_data2)):
    wr.writerow(import_data2[i][:])
w.close()

In [1]:
# 텐서플로우 모델 생성 위한 import

import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
tf.set_random_seed(777)

# 초기값 선정 xavier 알고리즘
def xavier_init(n_inputs, n_outputs, uniform=True):
    
    if uniform:
        init_range = tf.sqrt(6.0 / (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)
    else:
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)

# 단파장 비율 학습 및 테스트 (편차 분포)

import_data = np.genfromtxt('../data/swr_ratio_classification_input_am.csv', delimiter=',', dtype='float')

x_data = import_data[:85, :-1]
test_x = import_data[85:, :-1]

scaler = MinMaxScaler(feature_range=(0, 10))
x_data = scaler.fit_transform(x_data)

y_data = []
test_y = []

for i in range(85):
    temp = []
    temp.append(import_data[i][-1])
    y_data.append(temp)

raw_y = y_data
    
for i in range(85, len(import_data)):
    temp = []
    temp.append(import_data[i][-1])
    test_y.append(temp)

y_data = ohe.fit_transform(y_data)
y_data = y_data.toarray();

X = tf.placeholder("float", [None, 6])
Y = tf.placeholder("float", [None, 5])

nb_classes = 5

# W = tf.Variable(tf.random_normal([1, nb_classes]), name='weight')
# b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# Xavier Initializer 추가 코드
W = tf.get_variable("W", shape=[6, nb_classes], initializer=xavier_init(6, nb_classes))
b = tf.Variable(tf.zeros([nb_classes]))

H = tf.nn.softmax(tf.matmul(X, W) + b)

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(H), axis=1))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

with tf.Session() as sess:
    accuracy = 0
    sess.run(tf.global_variables_initializer())

    for step in range(11601):
        sess.run(optimizer, feed_dict={X:x_data, Y:y_data})
        if step % 100 == 0:
            print(step, sess.run(cost, feed_dict={X:x_data, Y:y_data}))

    print('-----------------------------')

    print('train_data = ', len(x_data), 'test_data = ', len(test_x))

    for i in range(len(test_x)):
        a = sess.run(H, feed_dict={X:[test_x[i]]})
        print(sess.run(tf.argmax(a, 1)), test_y[i])
#         print(a[0][0], ', ', a[0][1], ', ', a[0][2], ', ', a[0][3], ', ', a[0][4])
        if(sess.run(tf.argmax(a, 1)) ==test_y[i]):
            accuracy += 1
    print("accuracy = ", float(accuracy / len(test_x) * 100))

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected

0 11.052526
100 1.4523042
200 1.2206751
300 1.1181924
400 1.0615342
500 1.0214144
600 0.9902292
700 0.9652034
800 0.94483197
900 0.92807096
1000 0.91413397
1100 0.9024165
1200 0.8924559
1300 0.88389874
1400 0.8764775
1500 0.8699893
1600 0.8642796
1700 0.85922986
1800 0.8547468
1900 0.850757
2000 0.84720004
2100 0.8440261
2200 0.8411926
2300 0.83866334
2400 0.8364059
2500 0.8343923
2600 0.8325971
2700 0.8309974
2800 0.829573
2900 0.82830524
3000 0.8271776
3100 0.8261749
3200 0.82528365
3300 0.8244915
3400 0.82378775
3500 0.8231624
3600 0.82260686
3700 0.82211334
3800 0.82167494
3900 0.82128537
4000 0.82093936
4100 0.82063246
4200 0.82036
4300 0.8201186
4400 0.819905
4500 0.81971616
4600 0.8195499
4700 0.81940377
4800 0.8192756
4900 0.8191638
5000 0.8190666
5100 0.8189825
5200 0.81891006
5300 0.8188483
5400 0.8187956
5500 0.8187513
5600 0.8187141
5700 0.818683
5800 0.81865776
5900 0.8186383
6000 0.81862074
6100 0.81861323
6200 0.8186189
6300 0.81858915
6400 0.81858474
6500 0.8185781
6600